In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input{font-family:Consolas; font-size:12pt;}
div.prompt {min width:70px;}
div#toc-wrapper {padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe {font-size:12px;}
</style>
"""))

<font size='6' color='red'><b>ch2 Ollama LLM활용의 기본개념(LangChain)</b></font>
# 1. LLM을 활용하여 답변 생성하기
## 1) Ollama를 이용한 로컬 LLM 이용
- 성능은 GPT, CLaude 같은 모델보다 떨어지나, 개념설명을 위해 open source 모델 사용

### ollama.com 다운로드 -> 설치 -> 모델 pull
- cmd 창이나 powershell 창에 ollama run deepseek-r1:1.5b
https://docs.langchain.com/oss/python/integrations/chat/ollama

In [3]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="deepseek-r1:1.5b")
result = llm.invoke("What is the capital of Korea?")
result

AIMessage(content='The capital of Korea is釜山.', additional_kwargs={}, response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-12-09T02:05:46.3893471Z', 'done': True, 'done_reason': 'stop', 'total_duration': 546340700, 'load_duration': 87427200, 'prompt_eval_count': 10, 'prompt_eval_duration': 63311400, 'eval_count': 13, 'eval_duration': 382803700, 'logprobs': None, 'model_name': 'deepseek-r1:1.5b', 'model_provider': 'ollama'}, id='lc_run--019b00db-87f1-71c3-bb2b-0992cb31782c-0', usage_metadata={'input_tokens': 10, 'output_tokens': 13, 'total_tokens': 23})

### 모델 pull
- cmd나 powershell 창에서 : ollama pull llama3.2:1b

In [5]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model='llama3.2:1b')
result = llm.invoke("What is the capital of Korea?")
result

AIMessage(content="The capital of South Korea is Seoul. However, it's worth noting that North Korea also claims Seoul as its own capital, and many Koreans consider Mount Myohyang in North Korea to be their capital. But officially, Seoul remains the capital of South Korea.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T02:20:16.0904082Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3464817700, 'load_duration': 1272672500, 'prompt_eval_count': 32, 'prompt_eval_duration': 224774200, 'eval_count': 53, 'eval_duration': 1909547700, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b00e8-c1d1-79b2-bb15-927b7267aa58-0', usage_metadata={'input_tokens': 32, 'output_tokens': 53, 'total_tokens': 85})

In [6]:
result.content

"The capital of South Korea is Seoul. However, it's worth noting that North Korea also claims Seoul as its own capital, and many Koreans consider Mount Myohyang in North Korea to be their capital. But officially, Seoul remains the capital of South Korea."

In [7]:
result = llm.invoke("한국 수도가 어디에요?")
result.content

'한국의 수도는 Seoul입니다. 유한공리 church를 포함하여 한국에서 가장 큰 수도지로, 세계적으로 유명합니다.'

## 2) openai 활용
- pip install langchain-openai
- https://auth.openai.com/log-in

In [12]:
# 환경변수 가져오기
from dotenv import load_dotenv
import os
load_dotenv()

True

In [13]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-nano",
                #api_key=os.getenv("OPENAI_API_KEY") # api key를 OPENAI_API_KEY로 하지 않을경우에는 api_key를 지정해야한다
                )
result = llm.invoke("What is the capital of Korea? Return the name of the city only.")

In [14]:
result

AIMessage(content='Seoul', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 395, 'prompt_tokens': 21, 'total_tokens': 416, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CkkVyqbukGrnGYj4D3GEBaayuJqwI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0192-525c-79c2-b92d-4c8d6b238007-0', usage_metadata={'input_tokens': 21, 'output_tokens': 395, 'total_tokens': 416, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 384}})

In [ ]:
# Azure : OPENAI_API_VERSION키 값
# from langchain_openai import AzureOpenAI
# llm = AzureOpenAI(model="gpt-5-nano")

# 2. 랭체인 스타일로 프롬포트 작성
- 프롬프트  llm 호출시 쓰는 질문

In [25]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
#llm.invoke(0)
# 프롬프트로의 종류 : PromptValue, str, BaseMessages 리스트

## 1) 기본 프롬프트 템플릿 사용
- PromptTemplate을 사용하여 변수가 포함된 템플릿을 작성하면 PromptValue

In [22]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate(
    template="What is the capital of {country}?", # {}안의 값을 새로운 값으로 대입 가능
    input_variables=["country"]
)
country = input('어느 나라의 수도를 알고 싶으신가요?')
prompt = prompt_template.invoke({"country":country})
print(prompt)
llm.invoke(prompt)

어느 나라의 수도를 알고 싶으신가요?
text='What is the capital of ?'


AIMessage(content="Could you please provide more context or specify which country or location you're asking about? I'll do my best to provide the correct information.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T05:47:50.4240093Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1388801000, 'load_duration': 145781900, 'prompt_eval_count': 31, 'prompt_eval_duration': 162118400, 'eval_count': 29, 'eval_duration': 1049106200, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01a6-d3a9-7e11-8043-487e21ba9e78-0', usage_metadata={'input_tokens': 31, 'output_tokens': 29, 'total_tokens': 60})

## 2) 메세지 기반 프롬프트 작성
- BaseMessage 리스트
- BaseMessage 상속받은 클래스 : AIMessage, HumanMessage, SystemMessage, ToolMessage

In [4]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범질문
    AIMessage(content="The capital of Italy is Rome."), # 모범질문에 대한 모범답안
    HumanMessage(content="What is the capital of France?"), # 모범질문
    AIMessage(content="The capital of France is Paris."), # 모범질문에 대한 모범답안
    HumanMessage(content="What is the capital of Japan?"), # 모범질문
    AIMessage(content="The capital of Italy is Tokyo."), # 모범질문에 대한 모범답안
    HumanMessage(content="What is the capital of China?"), # 모범질문
    AIMessage(content="The capital of France is Beijing."), # 모범질문에 대한 모범답안
    HumanMessage(content="What is the capital of USA?"), # 모범질문
    AIMessage(content="The capital of Italy is Washington."), # 모범질문에 대한 모범답안
    HumanMessage(content="What is the capital of Canada?"), # 모범질문
    AIMessage(content="The capital of France is Ottawa."), # 모범질문에 대한 모범답안
    HumanMessage(content="What is the capital of England?")
]
llm.invoke(message_list)

AIMessage(content='The capital of the United Kingdom, which includes England, is London.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:12:33.3535489Z', 'done': True, 'done_reason': 'stop', 'total_duration': 841669300, 'load_duration': 157178900, 'prompt_eval_count': 182, 'prompt_eval_duration': 122316100, 'eval_count': 15, 'eval_duration': 541368700, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01bd-767d-7df2-8a00-1ddc91936563-0', usage_metadata={'input_tokens': 182, 'output_tokens': 15, 'total_tokens': 197})

## 3) ChatPromptTemplate 사용
- BaseMessage리스트 -> 튜플리스트

In [8]:
# 위의 BaseMessage 리스트를 수정
# PromptTemplate : 프롬프트에 변수포함,
# ChatPromptTemplate : SystemPrompt설정(페르소나), few shot 설정, 변수포함
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
chatPrompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant!"),
    ("human", "What is the capital of Italy?"),
    ("ai", "The capital of Italy is Rome."),
    ("human", "What is the capital of France?"),
    ("ai", "The capital of France is Paris."),
    ("human", "What is the capital of {country}?")
])

country = input("어느 나라 수도가 궁금하세요")
prompt = chatPrompt_template.invoke({"country": country})
# print('프롬프트 :', prompt, type(prompt))
result = llm.invoke(prompt)
result.content

어느 나라 수도가 궁금하세요Korea


'The capital of South Korea is Seoul. The capital of North Korea is Pyongyang.'

In [10]:
chatPrompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 대한민국 정보 전문 도우미입니다"),
    ("human", "{country}의 수도는 어디에요?")
])
country = input("어느나라 수도가 궁금하세요?")
prompt = chatPrompt_template.invoke({"country" : country})
# print(prompt)
result = llm.invoke(prompt)
print(result.content)

어느나라 수도가 궁금하세요?한국
한국의 수도는 Seoul로, 동서부에서 서北부를 통과하여 동쪽으로 방향을 가는 방정식으로 38° 37' north latitude와 126° 58' east longitude입니다.


In [11]:
result

AIMessage(content="한국의 수도는 Seoul로, 동서부에서 서北부를 통과하여 동쪽으로 방향을 가는 방정식으로 38° 37' north latitude와 126° 58' east longitude입니다.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:39:18.7509432Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2176248500, 'load_duration': 103250600, 'prompt_eval_count': 43, 'prompt_eval_duration': 168863100, 'eval_count': 50, 'eval_duration': 1850130200, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01d5-f05e-7791-801f-2a3e155d6d0a-0', usage_metadata={'input_tokens': 43, 'output_tokens': 50, 'total_tokens': 93})

# 3. 답변 형식 컨트롤하기
- llm.invoke()의 결과는 AIMessage() -> string이나 json, 객체 : OutputParser이용

## 1) 문자열 출력 파서 이용
- StrOutputParser를 사용하여 LLM 출력(AIMessage)를 단순 문자열로 변환

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
# 명시적인 지시사항이 포함된 프롬프트
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Return the name of the city only",
    input_variables = ["country"]
)

# 프롬프트 템플릿에 값 주입
prompt = prompt_template.invoke({"country" : "Korea"})
ai_message = llm.invoke(prompt)
# ai_message
#문자열 출력 파서를 이용하여 llm응답(AIMessage 객체)을 단순 문자열로 변환
output_parse = StrOutputParser()
result = output_parse.invoke(ai_message)
result

'Seoul'

In [16]:
output_parse.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

In [19]:
# 변수설정, system, few shot 지정
chat_prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant with expertise in South Korea"),
    ("human", "What is the capital of Italy?"),
    ("ai", "Rome."),
    ("human", "What is the capital of France?"),
    ("ai", "Paris."),
    ("human", "What is the capital of {country}?")
])

output_parse = StrOutputParser()
output_parse.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

## 2) Json 파서 이용
- {'name':'홍길동', 'age':22}

In [23]:
from langchain_core.output_parsers import JsonOutputParser
country_detail_prompt = PromptTemplate(
    template = """ Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and Json dictionary only
    """,
    input_variables=["country"]
)
prompt = country_detail_prompt.invoke({"country":"Korea"})
ai_message = llm.invoke(prompt)
# print(ai_message.content)
output_parser = JsonOutputParser()
json_result = output_parser.invoke(ai_message)
print(json_result)

{'capital': 'Seoul', 'population': 51000000, 'language': 'Korean', 'currency': 'South Korean won'}


In [25]:
output_parser.invoke(llm.invoke(country_detail_prompt.invoke({"country":"Japan"})))

{'title': 'Japan',
 'capital': 'Tokyo',
 'population': 128768500,
 'language': 'Japanese',
 'currency': {'symbol': '$', 'code': 'JPY'}}

## 3) 구조화된 출력 사용
- Pydantic 모델을 사용하여 LLM 출력을 구조화된 형식으로 받기 (JsonParser 보다 훨씬 안정적)
- Pydantic : 데이터 유효성검사, 설정관리를 간편하게 해주는 라이브러리

In [28]:
class User:
    def __init__(self, id_, name, is_active=True):
        self.id_ = id_
        self.name = name
        self.is_active = is_active
user = User(1, '홍길동')
print(user)

In [31]:
from pydantic import BaseModel, Field
class User(BaseModel):
    id:int = Field(gt=0, description="id")
    name:str = Field(min_length=2, description="name")
    is_active:bool = Field(default=True, description="id 활성화 여부")
user = User(id=1, name='홍길동')
print(user)

id=1 name='홍길동' is_active=True


In [33]:
country_detail_prompt = PromptTemplate(
    template = """ Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and Json dictionary only
    """,
    input_variables=["country"]
)
class CountryDetail(BaseModel): # description : 더 정확한 출력 유도
    capital:str = Field(description="the Capital of the country")
    population:int = Field(description="the population of the country")
    language:str = Field(description="the language of the country")
    currency:str = Field(description="the currency of the country")
structuredllm = llm.with_structured_output(CountryDetail)
info = structuredllm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
info

CountryDetail(capital='Seoul', population=51, language='Korean', currency='South Korean won')

In [34]:
info.capital, info.population, info.language, info.currency

('Seoul', 51, 'Korean', 'South Korean won')

In [35]:
print("info를 json으로 :", info.model_dump_json())

info를 json으로 : {"capital":"Seoul","population":51,"language":"Korean","currency":"South Korean won"}


# 4. LCEL을 활용한 렝체인 생성하기
## 1) 문자열 출력 파서 사용
- invoke
- StrOutputParser, ChatOllama, PromptTemplate등은 모두 XXX로 상속

In [36]:

from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="llama3.2:1b",
                temperature=0) # 일관된 답변(보수적인 답변)
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Retrun the name of the city only.",
    input_variables = ["country"]
)
output_parser = StrOutputParser() # AIMessage()를 Str변환
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

## 2) LCEL을 사용한 간단한 체인 구성
- 파이프 연산자(|) 이용

In [37]:
# 프롬프트 템플릿 -> llm -> 출력파서를 연결하는 체인 생성
capital_chain = prompt_template | llm | output_parser
# 생성된 체인 invoke
capital_chain.invoke({"country":"Korea"})

'Seoul'

## 3) 복합체인 구성
- 여러 단계의 추론이 필요한 경우 (체인 연결)

In [38]:
# 나라 설명 -> 나라 이름
country_prompt = PromptTemplate(
template = """ Guess the name of the country based on the following information:
    {information}
    Return the name of the country only""",
    input_variables=["information"])
output_parser.invoke(llm.invoke(country_prompt.invoke({"information": "Known for maple leaf"})))

'Canada.'

In [39]:
# 나라명 추출 체인 생성
country_chain = country_prompt | llm | output_parser
country_chain.invoke({"information": "Known for maple leaf"})

'Canada.'

In [40]:
# 복합체인 : 나라설명 -> 나라명 (country_chain)
#                     나라명 -> 수도 (capital chain)
final_chain = country_chain | capital_chain
final_chain.invoke({"information": "Known for maple leaf"})

'Ottawa'

In [41]:
# 복합체인 : information -> country_chain -> (나라명을 country) -> capital_chain
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information":RunnablePassthrough()} | {"country":country_chain} | capital_chain

In [42]:
final_chain.invoke("Known for maple leaf")

'Ottawa'

- 한글지원 여부 (한글 지원이 안 되는 모델은 렝체인 연결이 잘 안 됨)

In [43]:
# 나라 설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""다음의 {information} 설명을 보고 나라이름을 맞춰봐:
    {information}
    나라 이름만 한국어로 reutrn 해 줘""",
    input_variables=["information"]
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":
                            "이 나라는 와인으로 유명해"})))

'이 나라는 와인으로 유명해\n\n1. 프랑스 - burgundy, merlot, cabernet sauvignon\n2. 이탈리아 - chianti, pinot noir, vermentino\n3. 스타일러 - merlot, shiraz, syrah\n4. 이스라엘 - merlot, cabernet sauvignon, pinot noir\n5. 미국 - merlot, cabernet sauvignon, pinot noir'

# 5. 생성형 AI 평가 (생성형_AI.ipynb):
- 위 : 나라설명->나라이름->수도 (체인2개연결)
- 나라이름 -> 음식 -> 레시피

In [ ]:
## 복합체인 : 나라명 -> 그 나라에서 제일 유명한 음식 (food_chain)
                    # 음식 -> 레시피 (recipe_chain)